In [1]:
import os, sys, json
from dotenv import load_dotenv

load_dotenv()

print("OPENAI_API_KEY loaded?:", bool(os.getenv("OPENAI_API_KEY")))
print("SERPAPI_KEY loaded?:", bool(os.getenv("SERPAPI_KEY")))

OPENAI_API_KEY loaded?: True
SERPAPI_KEY loaded?: True


In [2]:
import requests
import os
import json

flight_request = {
    "origin": "JFK",
    "destination": "KTM",
    "outbound_date": "2025-12-10",
    "return_date": "2025-12-20",
    "adults": 2,
}

def fetch_flights(req):
    params = {
        "engine": "google_flights",
        "departure_id": req["origin"],
        "arrival_id": req["destination"],
        "outbound_date": req["outbound_date"],
        "return_date": req["return_date"],
        "adults": req["adults"],
        "currency": "USD",
        "api_key": os.getenv("SERPAPI_KEY"),
    }
    data = requests.get("https://serpapi.com/search", params=params).json()
    return data

raw_flights = fetch_flights(flight_request)

print("✅ Flight tool executed")
print("Routes found:", len(raw_flights.get("best_flights", [])))


✅ Flight tool executed
Routes found: 4


In [3]:
budget = 4000  # you can tweak this

def filter_budget(raw, budget):
    kept = []
    for item in raw.get("best_flights", []):
        if item.get("price", 0) <= budget:
            kept.append(item)
    return kept

budget_filtered = filter_budget(raw_flights, budget)

print("✅ Budget filter executed")
print("Flights under budget:", len(budget_filtered))


✅ Budget filter executed
Flights under budget: 3


In [8]:
def pick_top2(flights):
    """
    Sends flights to OpenAI and forces a JSON reply selecting top 2
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    prompt = f"""
    You are a flight ranking AI.
    Pick the best 2 flights based on:
    - cheaper first
    - fewer stops
    - shorter duration if similar price
    - ignore interests completely

    You MUST return ONLY valid JSON, nothing else:

    {{
      "origin": "{flight_request['origin']}",
      "destination": "{flight_request['destination']}",
      "outbound_date": "{flight_request['outbound_date']}",
      "return_date": "{flight_request['return_date']}",
      "flights": [
         {{"airline":"Example","departure_time":"YYYY-MM-DD HH:MM","arrival_time":"YYYY-MM-DD HH:MM","duration":123,"price":999,"stops":0}},
         {{"airline":"Example","departure_time":"YYYY-MM-DD HH:MM","arrival_time":"YYYY-MM-DD HH:MM","duration":123,"price":999,"stops":0}}
      ]
    }}

    Data to rank:
    {json.dumps(flights, indent=2)}
    """

    r = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role":"user","content":prompt}],
    )

    text = r.choices[0].message.content.strip()
    print("🧠 LLM response:\n", text)

    blob = re.search(r"\{.*\}", text, re.DOTALL)
    if not blob:
        raise ValueError("No JSON returned")
    
    return json.loads(blob.group(0))

optimized_top2 = pick_top2(budget_filtered)
print("\n✅ Optimized Top 2 JSON:\n", json.dumps(optimized_top2, indent=2))


🧠 LLM response:
 ```json
{
  "origin": "JFK",
  "destination": "HND",
  "outbound_date": "2025-12-10",
  "return_date": "2025-12-20",
  "flights": [
    {
      "airline": "American",
      "departure_time": "2025-12-10 07:59",
      "arrival_time": "2025-12-11 18:30",
      "duration": 1231,
      "price": 1945,
      "stops": 1
    },
    {
      "airline": "Air Canada",
      "departure_time": "2025-12-10 09:45",
      "arrival_time": "2025-12-11 16:40",
      "duration": 1015,
      "price": 2405,
      "stops": 1
    }
  ]
}
```

✅ Optimized Top 2 JSON:
 {
  "origin": "JFK",
  "destination": "HND",
  "outbound_date": "2025-12-10",
  "return_date": "2025-12-20",
  "flights": [
    {
      "airline": "American",
      "departure_time": "2025-12-10 07:59",
      "arrival_time": "2025-12-11 18:30",
      "duration": 1231,
      "price": 1945,
      "stops": 1
    },
    {
      "airline": "Air Canada",
      "departure_time": "2025-12-10 09:45",
      "arrival_time": "2025-12-11 16:40

In [9]:
def finalize_top2(opt_json):
    """
    Optionally re-pass into a clean object for later supervisor use
    """
    kept = opt_json["flights"][:2]
    return {"flights": kept}

final = finalize_top2(optimized_top2)
print("\n✈ Final 2 flights chosen:\n", json.dumps(final, indent=2))



✈ Final 2 flights chosen:
 {
  "flights": [
    {
      "airline": "American",
      "departure_time": "2025-12-10 07:59",
      "arrival_time": "2025-12-11 18:30",
      "duration": 1231,
      "price": 1945,
      "stops": 1
    },
    {
      "airline": "Air Canada",
      "departure_time": "2025-12-10 09:45",
      "arrival_time": "2025-12-11 16:40",
      "duration": 1015,
      "price": 2405,
      "stops": 1
    }
  ]
}
